In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #to be able to click/input
from selenium.webdriver.support.ui import WebDriverWait #wait excplicit
from selenium.webdriver.support import expected_conditions as EC #wait explicit
from selenium.webdriver.common.by import By #find_element
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from tabulate import tabulate
import pandas as pd
from pandas.errors import InvalidIndexError
import numpy as np

In [2]:
driver = webdriver.Chrome()

url = 'https://www.agoda.com/search?guid=072a0fac-d28e-4c65-a967-975833a22c48&asq=NQVGXW6jsE3tbdY9S%2BqUCpufa9Vwpz6XltTHq4n%2B9gPt6Sc9VYM%2BOtJvOdzFsuZ%2F3domjxbl67NdtXL0GBAVXaGQ1bwiOxIe%2FbY2DD%2FjB2ZjJ4jhAySpNAre7NeCVK9bATUmemHrs9SuRiKczbbzwwqQMycviD3CIWSJVwpQ8srbJXYAXDvNrCu7lWp2RsTjP54KjiP0bwIN%2BoYlOAPqpVur%2Ftc%2B54Iv4fnCDM4f8d8%3D&city=14018&tick=638616441062&locale=en-us&ckuid=9c386087-5783-4d15-8cee-affafdc523d8&prid=0&currency=IDR&correlationId=c56c8aaa-d549-4f28-840a-e0ea7c7dc2e7&analyticsSessionId=-1262129150739550036&pageTypeId=1&realLanguageId=1&languageId=1&origin=ID&stateCode=YO&cid=1844104&userId=9c386087-5783-4d15-8cee-affafdc523d8&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=25&currencyCode=IDR&htmlLanguage=en-us&cultureInfoName=en-us&machineName=sg-pc-6h-acm-web-user-7b4ddfb7f5-kw8zf&trafficGroupId=1&trafficSubGroupId=84&aid=130589&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Edge+%28Chromium%29+for+Windows&cdnDomain=agoda.net&checkIn=2024-09-17&checkOut=2024-09-18&rooms=1&adults=1&children=0&priceCur=IDR&los=1&textToSearch=Yogyakarta&travellerType=0&familyMode=off&ds=A0vbMurrcmdrV7DB&productType=-1' 

driver.get(url)

In [21]:
prog = 0
agoda_hotel = []

# Zoom level: 0.8 (might not be universal: try to jump down to the bottom of the page and zoom out until 'next' button is visible)
zoom_level = "0.8"
driver.execute_script(f"document.body.style.zoom='{zoom_level}'")

### ----- Loops ----- ###
while True:
    last_height = driver.execute_script("return window.pageYOffset + window.innerHeight")
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(0.5)
    new_height = driver.execute_script("return window.pageYOffset + window.innerHeight")
    if last_height == new_height:


### ----- Scraping ----- ###
        card = driver.find_elements(By.CSS_SELECTOR, 'ol.hotel-list-container')
        for x in card:
            hotel = x.find_elements(By.CSS_SELECTOR, 'li.PropertyCard.PropertyCardItem')
            for y in hotel:
                try:
                    # Name
                    name = y.find_element(By.CSS_SELECTOR, 'h3.sc-jrAGrp.sc-kEjbxe.eDlaBj.dscgss').text
                    
                    # Price
                    price = y.find_element(By.CSS_SELECTOR, 'div[data-element-name="final-price"] > span[data-selenium="display-price"]').text
                    
                    # Star
                    try:
                        star = y.find_elements(By.CSS_SELECTOR, 'svg.sc-eCssSg.sc-jSgupP.eUOibf')
                    except NoSuchElementException as idkwhybutihavetodothisforstar:
                        star = []
                    
                    # Appending
                    hotel_list = {'Nama Hotel': name,
                                  'Harga Hotel': price,
                                  'Bintang': len(star)
                                 }
                    agoda_hotel.append(hotel_list)
                    
                    # Progress Tracking
                    prog+=1
                    if prog % 25 == 0:
                        print(f'There are {prog} hotel information retreived')

                except NoSuchElementException as no_info:
                    continue
                    
#--- Begin the new Loops ---#
        next_page = driver.find_element(By.XPATH, '//*[@id="paginationNext"]')
        next_page.click()
        time.sleep(2)

There are 25 hotel information retreived
There are 50 hotel information retreived
There are 75 hotel information retreived
There are 100 hotel information retreived
There are 125 hotel information retreived
There are 150 hotel information retreived
There are 175 hotel information retreived
There are 200 hotel information retreived
There are 225 hotel information retreived
There are 250 hotel information retreived
There are 275 hotel information retreived
There are 300 hotel information retreived
There are 325 hotel information retreived
There are 350 hotel information retreived


KeyboardInterrupt: 

In [22]:
df = pd.DataFrame(agoda_hotel)

In [24]:
df

,Nama Hotel,Harga Hotel,Bintang
0,Amaya Suites Hotel,"140,637",2
1,RedDoorz Hostel near Stadion Mandala Krida Yogya,"42,972",2
2,Allstay Ecotel Yogyakarta,"226,436",2
3,The Cabin Hotel,"111,782",1
4,Grand Sarila Hotel Yogyakarta,"199,896",3
...,...,...,...
347,RedDoorz near Rumah Sakit Condong Catur,"120,284",2
348,RedDoorz Plus near Alun Alun Selatan 2,"140,862",3
349,Homestay C9 Exclusive Kaliurang Yogyakarta Mit...,"120,996",2
350,Joglo Plawang Villa & Resort,"393,930",4


In [27]:
df.to_csv("Agoda 17-18.09.24 Yogyakarta Hotel Pricelist.csv", index=False)

In [23]:
# New Code Learnt (to find): df[df.eq("Eastparc Hotel Yogyakarta").any(axis=1)]